In [1]:
import wandb
import datetime
import argparse
import numpy as np
from train_and_test import train_and_test
from helpers import get_NWP_data, get_rain_gauge_or_radar_data
from helpers import sw_func, find_experiment_directory, make_name_string
from helpers import use_old_forecasts_as_observations
from evaluator import only_MSE_and_GSS
import sys
from tensorflow import device

from models.model import model


# Fixed paramters
project = "Thesis Experiment 2 and 3"
d_start = datetime.datetime(2015, 1, 1)
d_end = datetime.datetime(2019, 12, 31, 23, 59, 59)
max_lag = 24
pred_dist = 60
t_roll = 24
threshold_dict = {
    "RVK-GEL": [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    "RVK-BOD": [0, 15,  30,  45,  60,  75,  90,  105, 120, 135, 150],
}
thresholds = np.array(list(threshold_dict.values()))
temperature_name = "1475_T"

experiment_dir = find_experiment_directory()
data_directory = f"{experiment_dir}/Data/"
save_dir = f"{experiment_dir}/predictions/"

early_stopper_params = {
    'monitor': 'val_loss',
    'min_delta': 0,
    'patience': 20,
    'verbose': 0,
    'mode': 'min',
    'baseline': None,
    'restore_best_weights': True,
    }

evaluation_config = {
    'eval_forecast_start': 24,
    'eval_forecast_end':30,
}


In [50]:
parser = argparse.ArgumentParser()
parser.add_argument("-r","--regularization",type=float,default=0.1,metavar="regularization",)
parser.add_argument("-u","--units",type=int,default=0,metavar="units",)
parser.add_argument("-f","--filters",type=int,default=0,metavar="filters",)
parser.add_argument("-l","--lr",type=float,default=0.1,metavar="lr",)
parser.add_argument("-w","--weights",type=str,default='a',metavar="lr",)
parser.add_argument("-b","--batch_size",type=int,default=256,metavar="batch_size",)
parser.add_argument("-d","--data_source",type=str,default='gauge',metavar="data_source",)
parser.add_argument("-o","--obs",type=int,default='3',metavar="obs",)
args = parser.parse_args([
                "--regularization",  "0.1",
                "--units",           "3",
                "--filters",         "0",
                "--lr",              "0.1",
                "--batch_size",      "8",
                "--data_source",     "radar",
                "--obs",             "4",
            ])
    

# Segja "Gauge" en nota radar

In [51]:

config_model = {
    'regularization':args.regularization,
    'units':args.units,
    'filters':args.filters,
    
    'lr': args.lr,
    'batch_size': args.batch_size,
    'data_source': args.data_source,
    'obs': args.obs,
    'using:':'gauge'
}
    
    
relative_weights = [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]
sample_weights_function = lambda x : sw_func(x, thresholds, relative_weights)


In [52]:

X1r,X1g,X2,X3,Y = get_rain_gauge_or_radar_data(threshold_dict,temperature_name,config_model['obs'],d_start,d_end)

In [53]:
radar_index = 215

config_model = {
    'regularization':args.regularization,
    'units':args.units,
    'filters':args.filters,
    
    'lr': args.lr,
    'batch_size': args.batch_size,
    'data_source': args.data_source,
    'obs': args.obs,
    'using:':'radar',
    'radar_index':radar_index
}

# 9 block mean:
# [:,:,[radar_index,radar_index+1,radar_index-1,
#                                    radar_index+20,radar_index+1+20,radar_index-1+20,
#                                    radar_index-20,radar_index+1-20,radar_index-1-20,]].mean(axis=-1)[...,np.newaxis]
    
X_radar = np.concatenate([X1r,X2, X3],axis=-1)
X = X_radar.astype('float32')
run = wandb.init(config=config_model, project='RadarVsGauge')
with device('cpu:0'):
    model_results = train_and_test(
        model_constructor=model_constructor,
        input_data=X,
        output_data=Y,
        weight_func=sample_weights_function,
        config=config_model,
    )
    
minimum_eval_dict = only_MSE_and_GSS(model_results["iop"],sample_weights_function,evaluation_config)
wandb.log(minimum_eval_dict['summary'])

epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,███████▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁
val_loss,█▁▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
best_epoch,1
best_val_loss,7595.32617
epoch,21
loss,9529.58008
lr,0.0027
val_loss,7916.41113


In [5]:
config_model = {
    'regularization':args.regularization,
    'units':args.units,
    'filters':args.filters,
    
    'lr': args.lr,
    'batch_size': args.batch_size,
    'data_source': args.data_source,
    'obs': args.obs,
    'using:':'gauge'
}
    

X_gauge = np.concatenate([X1g, X2, X3],axis=-1)

X = X_gauge.astype('float32')
Y = Y.reshape(-1,2).astype('float32')

model_constructor = lambda : model(config_model)
run = wandb.init(config=config_model, project='RadarVsGauge')
with device('cpu:0'):
    model_results = train_and_test(
        model_constructor=model_constructor,
        input_data=X,
        output_data=Y,
        weight_func=sample_weights_function,
        config=config_model,
    )
    
minimum_eval_dict = only_MSE_and_GSS(model_results["iop"],sample_weights_function,evaluation_config)
wandb.log(minimum_eval_dict['summary'])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sverrirhd (use `wandb login --relogin` to force relogin)
C:\Users\sverrirhd\Anaconda3\envs\thesis\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [16]:
radar_index = 215

config_model = {
    'regularization':args.regularization,
    'units':args.units,
    'filters':args.filters,
    
    'lr': args.lr,
    'batch_size': args.batch_size,
    'data_source': args.data_source,
    'obs': args.obs,
    'using:':'radar',
    'radar_index':radar_index
}

# 9 block mean:
# [:,:,[radar_index,radar_index+1,radar_index-1,
#                                    radar_index+20,radar_index+1+20,radar_index-1+20,
#                                    radar_index-20,radar_index+1-20,radar_index-1-20,]].mean(axis=-1)[...,np.newaxis]
    
X_radar = np.concatenate([X1r,X2, X3],axis=-1)
X = X_radar.astype('float32')
run = wandb.init(config=config_model, project='RadarVsGauge')
with device('cpu:0'):
    model_results = train_and_test(
        model_constructor=model_constructor,
        input_data=X,
        output_data=Y,
        weight_func=sample_weights_function,
        config=config_model,
    )
    
minimum_eval_dict = only_MSE_and_GSS(model_results["iop"],sample_weights_function,evaluation_config)
wandb.log(minimum_eval_dict['summary'])

GSS_val_0,▁
GSS_val_1,▁
GSS_val_average,▁
epoch,▁▂▂▃▄▅▅▆▇▁▂▂▃▄▅▅▆▇▇█▂▂▃▄▁▂▃▃▄▅▅▁▂▃▃▄▅▆▆▇
loss,█▄▃▂▂▁▁▁▁▇▂▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁
lr,███████▃▂█████▃▃▂▁▁▁██▃▂███▃▃▂▁██▃▃▂▂▂▁▁
total_mean_absolute_error,▁
total_mean_squared_error,▁
total_weighted_mean_absolute_error,▁
total_weighted_mean_squared_error,▁
val_loss,█▅▃▂▂▂▂▂▂█▃▂▂▂▂▂▂▂▂▂▃▃▃▃▆▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁


KeyboardInterrupt: 

# A few random pixels around

In [ ]:
model_results

In [ ]:

# run = wandb.init(config=config_model, project=project)
# with device('cpu:0'):


predictions = {fold:{'Y_val_pred':model_results['iop'][fold]['Y_val_pred'].astype('float32'),
                        'Y_train_pred':model_results['iop'][fold]['Y_train_pred'].astype('float32')} for fold in range(5)}
file_name = make_name_string('Predictions',config_model)
np.save(save_dir + file_name,predictions, allow_pickle=True)

minimum_eval_dict = only_MSE_and_GSS(model_results["iop"],sample_weights_function,evaluation_config)
wandb.log(minimum_eval_dict['summary'])
run.finish()
